In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['wikisql', 'hindienglish-corpora']


In [2]:
lines=pd.read_csv("../input/wikisql/train.csv",encoding='utf-8')

In [3]:
#lines['source'].value_counts()

In [4]:
#lines=lines[lines['source']=='ted']

In [5]:
lines.head(20)

,question,sql
0,Tell me what the notes are for South Australia,SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA
1,What is the current series where the new series began in June 2011?,SELECT Current series FROM table WHERE Notes = New series began in June 2011
2,What is the format for South Australia?,SELECT Format FROM table WHERE State/territory = South Australia
3,Name the background colour for the Australian Capital Territory,SELECT Text/background colour FROM table WHERE State/territory = Australian Capital Territory
4,how many times is the fuel propulsion is cng?,SELECT COUNT Fleet Series (Quantity) FROM table WHERE Fuel Propulsion = CNG
5,what is the fuel propulsion where the fleet series (quantity) is 310-329 (20)?,SELECT Fuel Propulsion FROM table WHERE Fleet Series (Quantity) = 310-329 (20)
6,who is the manufacturer for the order year 1998?,SELECT Manufacturer FROM table WHERE Order Year = 1998
7,how many times is the model ge40lfr?,SELECT COUNT Manufacturer FROM table WHERE Model = GE40LFR
8,how many times is the fleet series (quantity) is 468-473 (6)?,SELECT COUNT Order Year FROM table WHERE Fleet Series (Quantity) = 468-473 (6)
9,what is the powertrain (engine/transmission) when the order year is 2000?,SELECT Powertrain (Engine/Transmission) FROM table WHERE Order Year = 2000


In [6]:
lines.rename(columns = {'question':'english_sentence', 'sql':'hindi_sentence'}, inplace = True)

pd.isnull(lines).sum()

english_sentence    0
hindi_sentence      0
dtype: int64

In [7]:
# lines=lines[~pd.isnull(lines['english_sentence'])]

In [8]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [9]:
lines=lines.sample(n=35000,random_state=42)
lines.shape

(35000, 2)

In [10]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [11]:
# Remove quotes
#lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
#lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [12]:
#exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
#lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
#lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [13]:
# Remove all numbers from text
#remove_digits = str.maketrans('', '', digits)
#lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
#lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

#lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [14]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
lines.head()

,english_sentence,hindi_sentence
43648,how much is the purse worth (¥) after 2012?,START_ select sum purse (¥) from table where year > 2012 _END
46173,what is the name for malay with a foundation in iptura?,START_ select name in malay from table where foundation = iptura _END
19352,what is the highest weight for the farnley stakes race?,START_ select max weight (kg) from table where race = farnley stakes _END
3130,what are the won games with losing bonus of 0?,START_ select won from table where losing bonus = 0 _END
3192,what's the airport with % change 2005/2006 being 13.0%,START_ select airport from table where % change 2005/2006 = 13.0% _END


In [16]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [17]:
len(all_eng_words)

38969

In [18]:
len(all_hindi_words)

30712

In [19]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [20]:
lines.head()

,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
43648,how much is the purse worth (¥) after 2012?,START_ select sum purse (¥) from table where year > 2012 _END,9,12
46173,what is the name for malay with a foundation in iptura?,START_ select name in malay from table where foundation = iptura _END,11,12
19352,what is the highest weight for the farnley stakes race?,START_ select max weight (kg) from table where race = farnley stakes _END,10,13
3130,what are the won games with losing bonus of 0?,START_ select won from table where losing bonus = 0 _END,10,11
3192,what's the airport with % change 2005/2006 being 13.0%,START_ select airport from table where % change 2005/2006 = 13.0% _END,5,12


In [21]:
lines[lines['length_eng_sentence']>30].shape

(75, 4)

In [22]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [23]:
lines.shape

(32775, 4)

In [24]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [25]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [26]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(38969, 30712)

In [27]:
num_decoder_tokens += 1 #for zero padding


In [28]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [29]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [30]:
lines = shuffle(lines)
lines.head(30)

,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
5279,"at a longitude of 293.0e, what is the diameter, in km?",START_ select diameter (km) from table where longitude = 293.0e _END,11,11
12972,when موسم زيتون is the arabic title when is the year (ceremony)?,START_ select year (ceremony) from table where arabic title = موسم زيتون _END,12,13
36710,which college is the player from south medford high school headed to?,START_ select college from table where school = south medford high school _END,12,13
14762,how many college/junior/club teams are from sweden?,START_ select count college/junior/club team from table where nationality = sweden _END,7,12
34773,what is the score when the venue is athens olympic stadium on 30 april 2011?,START_ select score from table where venue = athens olympic stadium and year = 30 april 2011 _END,15,18
24722,what points average has a played greater than 20?,START_ select avg points from table where played > 20 _END,9,11
21306,what win method has a time of 2:32?,START_ select method from table where time = 2:32 _END,8,10
48460,which 2010/ 11 has a 2011/ 12 of wr?,START_ select 2010/ 11 from table where 2011/ 12 = wr _END,9,12
53815,what is the molecular target of the organism that has a disease area of antiviral?,START_ select molecular target from table where disease area = antiviral _END,15,12
43896,who was the winning team on july 13?,START_ select winning team from table where date = july 13 _END,8,12


### Split the data into train and test

In [31]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((26220,), (6555,))

### Let us save this data

In [32]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [33]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [34]:
latent_dim=300

In [35]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [36]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [37]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [38]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    11690700    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    9213900     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [39]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 20

In [40]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
409/409 [==============================] - 157s 384ms/step - loss: 4.0024 - val_loss: 3.3717
Epoch 2/20
409/409 [==============================] - 152s 373ms/step - loss: 3.1081 - val_loss: 3.0581
Epoch 3/20
409/409 [==============================] - 152s 371ms/step - loss: 2.7318 - val_loss: 2.8534
Epoch 4/20
409/409 [==============================] - 151s 369ms/step - loss: 2.4454 - val_loss: 2.7011
Epoch 5/20
409/409 [==============================] - 151s 370ms/step - loss: 2.2340 - val_loss: 2.6092
Epoch 6/20
409/409 [==============================] - 153s 374ms/step - loss: 2.0680 - val_loss: 2.5427
Epoch 7/20
409/409 [==============================] - 153s 374ms/step - loss: 1.9291 - val_loss: 2.4921
Epoch 8/20
409/409 [==============================] - 153s 374ms/step - loss: 1.8129 - val_loss: 2.4440
Epoch 9/20
409/409 [======================

In [41]:
model.save_weights('nmt_weights.h5')

In [42]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    11690700    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    9213900     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [43]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [44]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [45]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [46]:
f_question_file = open("questionenglish***.txt","w")
sql_file = open("sql**.txt","w")
predection_file = open("predection***.txt","w")

In [47]:
# f_question_file = open("questionenglish.txt","r")
# sql_file = open("sql.txt","r")
# predection_file = open("predection.txt","r")
# print([i for i in f_question_file.readlines()])
# print('*'*50)
# print([i for i in sql_file.readlines()])
# print('*'*50)
# print([i for i in predection_file.readlines()])

# # print(sql_file)

In [48]:
#X_train
#y_train


df = pd.DataFrame(columns = ['Input qustion sentence', 'Actual sql Translation', 'Prideccted'])
for x in range(10000):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    #print('Input English sentence:', X_train[k:k+1].values[0])
    #print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
    #print('Predicted Hindi Translation:', decoded_sentence[:-4])
    data_dict = {
        "Input qustion sentence": X_train[k:k+1].values[0],
        "Actual sql Translation": y_train[k:k+1].values[0][6:-4],
        "Prideccted": decoded_sentence[:-4]
    }
    
    df = df.append(data_dict,ignore_index = True)
    
    
    f_question_file = open("questionenglish***.txt","a")
    sql_file = open("sql***.txt","a")
    predection_file = open("predection***.txt","a")

    f_question_file.write(X_train[k:k+1].values[0]+'\n')
    sql_file.write(y_train[k:k+1].values[0][6:-4]+'\n')
    predection_file.write(decoded_sentence[:-4]+'\n')
    
    
    # =X_train[k:k+1].values[0]
    #rint(t)
    #ist_str=[]
    #ist_str.append(t)
    #rint(list_str) 
    #f = pd.DataFrame({"Input qustion sentence": list_str})
    #     print('-------------------------------------')


    f_question_file.close()
    sql_file.close()
    predection_file.close()
df.to_csv('resultMachine.csv')


In [51]:
res = pd.read_csv("resultMachine.csv")

NameError: name 'false' is not defined

In [50]:
res.head()

,Unnamed: 0,Input qustion sentence,Actual sql Translation,Prideccted
0,0,what are the notes when the time was 7:42.92 and the rank is more than 1?,select notes from table where rank > 1 and time = 7:42.92,select notes from table where rank > 1 and time =
1,1,name the away team score for home team of university,select away team score from table where home team = university,select away team score from table where home te
2,2,which venue had a nationality of belgium?,select venue from table where nationality = belgium,select venue from table where nationality = sw
3,3,which rank is the highest one that has points smaller than 256.6?,select max rank from table where points < 256.6,select max rank from table where points < 49
4,4,"which result has an attendance larger than 38,354, and a date of october 23, 1988?","select result from table where attendance > 38,354 and date = october 23, 1988",select result from table where attendance > > 2
